In [1]:
import os
import numpy as np
import torch
import time
import pickle
import matplotlib.pyplot as plt
from collections import OrderedDict
from torch.autograd import Variable
from pathlib import Path

%matplotlib inline

In [2]:
pix2pixhd_dir = Path('../src/pix2pixHD/')

import sys
sys.path.append(str(pix2pixhd_dir))

%load_ext autoreload
%autoreload 2

In [3]:
from options.train_options import TrainOptions
from data.data_loader import CreateDataLoader
from models.models import create_model
import util.util as util
from util.visualizer import Visualizer

In [4]:
with open('../data/train_opt.pkl', mode='rb') as f:
    opt = pickle.load(f)
    
iter_path = os.path.join(opt.checkpoints_dir, opt.name, 'iter.txt')

In [5]:
#opt.lr = 7e-4
opt.loadSize = 224
# opt.resize_or_crop = None
opt.no_flip = True
opt.nThreads = 2
#opt.lambda_feat = 25.0
opt.model = 'pix2pixHD_faceGAN'
opt.dataroot='../data/target/MJ_dance/'
opt.name = 'MJ_dance'
opt.add_face_disc = True
opt.fineSize = opt.loadSize
opt.num_D = 3
# opt.n_local_enhancers = 0
# opt.n_layers_D = 1
# opt.lr = 5e-5
opt.temp_data = False
#opt.no_ganFeat_loss = True
#opt.niter=10
#opt.niter_decay=10
opt.label_nc = 25
opt.debug = True
#opt.load_pretrain = '../checkpoints/target/'

In [6]:
if opt.continue_train:
    try:
        start_epoch, epoch_iter = np.loadtxt(iter_path , delimiter=',', dtype=int)
    except:
        start_epoch, epoch_iter = 1, 0
    print('Resuming from epoch %d at iteration %d' % (start_epoch, epoch_iter))        
else:    
    start_epoch, epoch_iter = 1, 0

if opt.debug:
    opt.display_freq = 1
    opt.print_freq = 1
    opt.niter = 5
    opt.niter_decay = 5
    opt.max_dataset_size = 10
opt

Namespace(add_face_disc=True, batchSize=1, beta1=0.5, checkpoints_dir='../checkpoints/', continue_train=False, data_type=32, dataroot='../data/target/MJ_dance/', debug=True, display_freq=1, display_winsize=512, feat_num=3, fineSize=224, fine_size=480, gpu_ids=[0], input_nc=3, instance_feat=False, isTrain=True, label_feat=False, label_nc=25, lambda_feat=10.0, loadSize=224, load_features=False, load_pretrain='', lr=0.0002, max_dataset_size=10, model='pix2pixHD_faceGAN', nThreads=2, n_blocks_global=9, n_blocks_local=3, n_clusters=10, n_downsample_E=4, n_downsample_global=4, n_layers_D=3, n_local_enhancers=1, name='MJ_dance', ndf=64, nef=16, netG='global', ngf=64, niter=5, niter_decay=5, niter_fix_global=0, no_flip=True, no_ganFeat_loss=False, no_html=False, no_instance=True, no_lsgan=False, no_vgg_loss=False, norm='instance', num_D=3, output_nc=3, phase='train', pool_size=0, print_freq=1, resize_or_crop='scale_width', save_epoch_freq=10, save_latest_freq=640, serial_batches=False, temp_da

In [7]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
print('#training images = %d' % dataset_size)
    
# start_epoch, epoch_iter = 1, 0
total_steps = (start_epoch-1) * dataset_size + epoch_iter
display_delta = total_steps % opt.display_freq
print_delta = total_steps % opt.print_freq
save_delta = total_steps % opt.save_latest_freq

CustomDatasetDataLoader
dataset [AlignedDataset] was created
#training images = 10


In [8]:
model = create_model(opt)
visualizer = Visualizer(opt)

GlobalGenerator(
  (model): Sequential(
    (0): ReflectionPad2d((3, 3, 3, 3))
    (1): Conv2d(25, 64, kernel_size=(7, 7), stride=(1, 1))
    (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (3): ReLU(inplace)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (6): ReLU(inplace)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (9): ReLU(inplace)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (11): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (12): ReLU(inplace)
    (13): Conv2d(512, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (14): InstanceNorm2d(1024, eps=1e-05, momentum=0.1, affine=False, track_run

create web directory ../checkpoints/MJ_dance/web...


In [9]:
for epoch in range(start_epoch, opt.niter + opt.niter_decay + 1):
    epoch_start_time = time.time()
    if epoch != start_epoch:
        epoch_iter = epoch_iter % dataset_size
    for i, data in enumerate(dataset, start=epoch_iter):
        iter_start_time = time.time()
        total_steps += opt.batchSize
        epoch_iter += opt.batchSize

        # whether to collect output images
        save_fake = total_steps % opt.display_freq == display_delta
        
        ############## Forward Pass ######################
        losses, generated = model(Variable(data['label']), Variable(data['inst']), 
            Variable(data['image']), Variable(data['feat']), infer=save_fake)
        
        # sum per device losses
        losses = [ torch.mean(x) if not isinstance(x, int) else x for x in losses ]
        loss_dict = dict(zip(model.module.loss_names, losses))

        # calculate final loss scalar
        loss_D = (loss_dict['D_fake'] + loss_dict['D_real']) * 0.5 
        loss_D += (loss_dict['D_f_real'] + loss_dict['D_f_fake']) * 0.5 
        loss_G = loss_dict['G_GAN'] + loss_dict.get('G_GAN_Feat',0) + loss_dict.get('G_VGG',0)
        loss_G += loss_dict['G_f_GAN'] + loss_dict.get('G_f_GAN_Feat', 0)
        # 'G_f_GAN', 'G_f_GAN_Feat', 'D_f_real', 'D_f_fake'
        ############### Backward Pass ####################
        # update generator weights
        model.module.optimizer_G.zero_grad()
        loss_G.backward()
        model.module.optimizer_G.step()

        # update discriminator weights
        model.module.optimizer_D.zero_grad()
        loss_D.backward()
        model.module.optimizer_D.step()
        
        #call(["nvidia-smi", "--format=csv", "--query-gpu=memory.used,memory.free"]) 

        ############## Display results and errors ##########
        ### print out errors
        if total_steps % opt.print_freq == print_delta:
            errors = {k: v.data[0] if not isinstance(v, int) else v for k, v in loss_dict.items()}
            t = (time.time() - iter_start_time) / opt.batchSize
            visualizer.print_current_errors(epoch, epoch_iter, errors, t)
            visualizer.plot_current_errors(errors, total_steps)

        ### display output images
        if save_fake:
            input_image =  util.tensor2im(data['image'][0])
            if opt.add_face_disc:
                    ys, ye, xs, xe = model.module.get_face_region(data['label'])
                    if ys is not None:
                        input_image[ys, xs:xe, :] = input_image[ye, xs:xe, :] = input_image[ys:ye, xs, :] = input_image[ys:ye, xe, :] = 255 

            visuals = OrderedDict([('input_label', util.tensor2label(data['label'][0], opt.label_nc)),
                                   ('synthesized_image', util.tensor2im(generated[0].data)),
                                   # ('res_image', util.tensor2im(generated[1].data[0])),
                                   ('real_image', input_image)])
                                   #('feat_image', util.tensor2im(data['feat'][0]))])
            visualizer.display_current_results(visuals, epoch, total_steps)

        ### save latest model
        if total_steps % opt.save_latest_freq == save_delta:
            print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            model.module.save('latest')            
            np.savetxt(iter_path, (epoch, epoch_iter), delimiter=',', fmt='%d')

        if epoch_iter >= dataset_size:
            break
       
    # end of epoch 
    iter_end_time = time.time()
    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))

    ### save model for this epoch
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))        
        model.module.save('latest')
        model.module.save(epoch)
        np.savetxt(iter_path, (epoch+1, 0), delimiter=',', fmt='%d')

    ### instead of only training the local enhancer, train the entire network after certain iterations
    if (opt.niter_fix_global != 0) and (epoch == opt.niter_fix_global):
        model.module.update_fixed_params()

    ### linearly decay learning rate after certain iterations
    if epoch > opt.niter:
        model.module.update_learning_rate()
        
torch.cuda.empty_cache()

/home/yaosy/anaconda3/envs/pytorch/lib/python3.6/site-packages/ipykernel_launcher.py:43: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


(epoch: 1, iters: 1, time: 2.744) G_GAN: 8.321 G_GAN_Feat: 16.666 G_VGG: 14.179 D_real: 8.611 D_fake: 3.658 G_f_GAN: 8.633 G_f_GAN_Feat: 33.112 D_f_real: 4.009 D_f_fake: 2.530 
(epoch: 1, iters: 2, time: 0.262) G_GAN: 7.723 G_GAN_Feat: 16.566 G_VGG: 13.777 D_real: 9.744 D_fake: 6.269 G_f_GAN: 7.655 G_f_GAN_Feat: 31.692 D_f_real: 3.965 D_f_fake: 3.451 
(epoch: 1, iters: 3, time: 0.280) G_GAN: 8.038 G_GAN_Feat: 16.023 G_VGG: 14.064 D_real: 9.659 D_fake: 8.435 G_f_GAN: 4.841 G_f_GAN_Feat: 32.259 D_f_real: 3.270 D_f_fake: 2.651 
(epoch: 1, iters: 4, time: 0.257) G_GAN: 5.744 G_GAN_Feat: 17.133 G_VGG: 14.073 D_real: 4.168 D_fake: 6.183 G_f_GAN: 4.339 G_f_GAN_Feat: 34.286 D_f_real: 2.687 D_f_fake: 1.971 
(epoch: 1, iters: 5, time: 0.265) G_GAN: 5.644 G_GAN_Feat: 16.337 G_VGG: 13.694 D_real: 4.338 D_fake: 5.781 G_f_GAN: 5.206 G_f_GAN_Feat: 31.646 D_f_real: 1.883 D_f_fake: 2.629 
(epoch: 1, iters: 6, time: 0.266) G_GAN: 4.026 G_GAN_Feat: 15.664 G_VGG: 13.355 D_real: 2.980 D_fake: 3.917 G_f_GAN

(epoch: 5, iters: 7, time: 0.232) G_GAN: 1.817 G_GAN_Feat: 10.129 G_VGG: 12.407 D_real: 0.766 D_fake: 1.008 G_f_GAN: 1.931 G_f_GAN_Feat: 14.185 D_f_real: 1.014 D_f_fake: 0.962 
(epoch: 5, iters: 8, time: 0.236) G_GAN: 2.621 G_GAN_Feat: 10.089 G_VGG: 11.587 D_real: 0.832 D_fake: 0.601 G_f_GAN: 3.646 G_f_GAN_Feat: 17.675 D_f_real: 1.702 D_f_fake: 1.711 
(epoch: 5, iters: 9, time: 0.231) G_GAN: 1.575 G_GAN_Feat: 10.404 G_VGG: 12.353 D_real: 0.609 D_fake: 1.040 G_f_GAN: 3.046 G_f_GAN_Feat: 15.458 D_f_real: 1.163 D_f_fake: 1.545 
(epoch: 5, iters: 10, time: 0.234) G_GAN: 2.021 G_GAN_Feat: 10.744 G_VGG: 12.469 D_real: 0.761 D_fake: 0.851 G_f_GAN: 2.216 G_f_GAN_Feat: 19.563 D_f_real: 1.206 D_f_fake: 0.776 
End of epoch 5 / 10 	 Time Taken: 3 sec
(epoch: 6, iters: 1, time: 0.231) G_GAN: 2.278 G_GAN_Feat: 9.912 G_VGG: 11.727 D_real: 0.789 D_fake: 0.734 G_f_GAN: 1.963 G_f_GAN_Feat: 16.860 D_f_real: 0.883 D_f_fake: 0.694 
(epoch: 6, iters: 2, time: 0.230) G_GAN: 2.126 G_GAN_Feat: 8.698 G_VGG: 11.

(epoch: 10, iters: 4, time: 0.227) G_GAN: 1.675 G_GAN_Feat: 8.298 G_VGG: 10.374 D_real: 0.393 D_fake: 0.381 G_f_GAN: 1.651 G_f_GAN_Feat: 12.678 D_f_real: 0.537 D_f_fake: 0.397 
(epoch: 10, iters: 5, time: 0.224) G_GAN: 1.830 G_GAN_Feat: 9.163 G_VGG: 9.983 D_real: 0.701 D_fake: 0.411 G_f_GAN: 1.288 G_f_GAN_Feat: 11.092 D_f_real: 0.546 D_f_fake: 0.718 
(epoch: 10, iters: 6, time: 0.225) G_GAN: 1.664 G_GAN_Feat: 7.792 G_VGG: 9.807 D_real: 0.410 D_fake: 0.420 G_f_GAN: 1.661 G_f_GAN_Feat: 12.246 D_f_real: 0.550 D_f_fake: 0.516 
(epoch: 10, iters: 7, time: 0.223) G_GAN: 1.505 G_GAN_Feat: 6.975 G_VGG: 9.848 D_real: 0.398 D_fake: 0.441 G_f_GAN: 1.373 G_f_GAN_Feat: 9.908 D_f_real: 0.606 D_f_fake: 0.543 
(epoch: 10, iters: 8, time: 0.223) G_GAN: 1.679 G_GAN_Feat: 8.505 G_VGG: 10.404 D_real: 0.404 D_fake: 0.418 G_f_GAN: 1.585 G_f_GAN_Feat: 12.640 D_f_real: 0.517 D_f_fake: 0.525 
(epoch: 10, iters: 9, time: 0.224) G_GAN: 1.714 G_GAN_Feat: 7.686 G_VGG: 9.769 D_real: 0.483 D_fake: 0.432 G_f_GAN: 1.6

In [10]:
data['label'].shape

torch.Size([1, 1, 224, 224])

In [11]:
losses

[tensor(1.6665, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(8.2942, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(10.0581, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(0.4408, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(0.4475, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(1.4475, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(13.7089, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(0.7248, device='cuda:0', grad_fn=<MeanBackward1>),
 tensor(0.5185, device='cuda:0', grad_fn=<MeanBackward1>)]

In [12]:
data['image'].size()

torch.Size([1, 3, 224, 224])

In [13]:
data['label'].size()

torch.Size([1, 1, 224, 224])

In [14]:
data['feat'].size()

torch.Size([1])

In [15]:
768 / 32

24.0

In [16]:
import cv2
import numpy as np

In [17]:
img = cv2.imread('/home/yaosy/Document/club/deep-imitation-dev/server-backend/data/target/MJ_dance/train_img/img_4508.png')

In [18]:
np.zeros(img.shape[:2]).shape

(480, 480)